## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?

In [1]:
!mlflow --version

mlflow, version 2.13.0


C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\_internal\_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


## Q2. Download and preprocess the data
We'll use the Green Taxi Trip Records dataset to predict the duration of each trip.

Download the data for January, February and March 2023 in parquet format from here.

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will:

- load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
- fit a DictVectorizer on the training set (January 2023 data),
- save the preprocessed datasets and the DictVectorizer to disk.
Your task is to download the datasets and then execute this command:

```python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output```

Tip: go to 02-experiment-tracking/homework/ folder before executing the command and change the value of <TAXI_DATA_FOLDER> to the location where you saved the data.

How many files were saved to OUTPUT_FOLDER?

In [6]:
# Get the dataset

!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet -P ./data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet -P ./data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet -P ./data

--2024-05-24 23:17:09--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.38.147, 18.239.38.83, 18.239.38.163, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.38.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: './data/green_tripdata_2023-01.parquet.1'

     0K .......... .......... .......... .......... ..........  3%  191K 7s
    50K .......... .......... .......... .......... ..........  7%  473K 5s
   100K .......... .......... .......... .......... .......... 10%  359K 4s
   150K .......... .......... .......... .......... .......... 14%  603K 4s
   200K .......... .......... .......... .......... .......... 17%  780K 3s
   250K .......... .......... .......... .......... .......... 21% 1.07M 3s
   300K .......... .......... .......... .......... ..........

In [7]:
!python preprocess_data.py --raw_data_path ./data --dest_path ./output

In [9]:
import os
os.listdir("./output")

['dv.pkl', 'test.pkl', 'train.pkl', 'val.pkl']

**4 Files** in output folder

## Q3. Train a model with autolog
We will train a RandomForestRegressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework.

The script will:

- load the datasets produced by the previous step,
- train the model on the training set,
- calculate the RMSE score on the validation set.
Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

In [11]:
!python train.py

2024/05/24 23:19:53 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/24 23:20:11 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."
C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [15]:
from mlflow.tracking import MlflowClient

TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=TRACKING_URI)

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [16]:
client.search_experiments()


[<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1716563281836, experiment_id='3', last_update_time=1716563281836, lifecycle_stage='active', name='random-forest-best-models', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1716561755926, experiment_id='2', last_update_time=1716561755926, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='file:///C:/Users/crab/py2/mlops-zoomcamp/cohorts/2024/02-experiment-tracking/homework/mlruns/1', creation_time=1716561207595, experiment_id='1', last_update_time=1716561207595, lifecycle_stage='active', name='homework2', tags={}>,
 <Experiment: artifact_location='file:///C:/Users/crab/py2/mlops-zoomcamp/cohorts/2024/02-experiment-tracking/homework/mlruns/0', creation_time=1716561207590, experiment_id='0', last_update_time=1716561207590, lifecycle_stage='active', name='Default', tags={}>]

In [47]:
client.search_runs(experiment_ids=1)

[<Run: data=<RunData: metrics={'training_mean_absolute_error': 3.4244701942312354,
  'training_mean_squared_error': 27.083054499499358,
  'training_r2_score': 0.6673983775155525,
  'training_root_mean_squared_error': 5.204138209108148,
  'training_score': 0.6673983775155525}, params={'bootstrap': 'True',
  'ccp_alpha': '0.0',
  'criterion': 'squared_error',
  'max_depth': '10',
  'max_features': '1.0',
  'max_leaf_nodes': 'None',
  'max_samples': 'None',
  'min_impurity_decrease': '0.0',
  'min_samples_leaf': '1',
  'min_samples_split': '2',
  'min_weight_fraction_leaf': '0.0',
  'monotonic_cst': 'None',
  'n_estimators': '100',
  'n_jobs': 'None',
  'oob_score': 'False',
  'random_state': '0',
  'verbose': '0',
  'warm_start': 'False'}, tags={'estimator_class': 'sklearn.ensemble._forest.RandomForestRegressor',
  'estimator_name': 'RandomForestRegressor',
  'mlflow.log-model.history': '[{"run_id": "50a15d1138c34be6a2fd613a086a2f88", '
                              '"artifact_path": "mo

In [48]:
run_id = "50a15d1138c34be6a2fd613a086a2f88"

run = client.get_run(run_id=run_id)

In [49]:
run.data.params.get('min_samples_split', None)

'2'

What is the value of the min_samples_split parameter:

**2**


## Q4. Launch the tracking server locally
Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to:

- launch the tracking server on your local machine,
- select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?
 

In [51]:
!mlflow server --help

Usage: mlflow server [OPTIONS]

  Run the MLflow tracking server.

  The server listens on http://localhost:5000 by default and only accepts
  connections from the local machine. To let the server accept connections
  from other machines, you will need to pass ``--host 0.0.0.0`` to listen on
  all network interfaces (or a specific interface address).

Options:
  --backend-store-uri PATH        URI to which to persist experiment and run
                                  data. Acceptable URIs are SQLAlchemy-
                                  compatible database connection strings (e.g.
                                  'sqlite:///path/to/file.db') or local
                                  filesystem URIs (e.g.
                                  'file:///absolute/path/to/directory'). By
                                  default, data will be logged to the ./mlruns
                                  directory.
  --registry-store-uri URI        URI to which to persist registered models.
    

C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\_internal\_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

**--default-artifact-root**

## Q5. Tune model hyperparameters
Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. We have prepared the script `hpo.py` for this exercise.

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

the list of hyperparameters that are passed to the `objective` function during the optimization,
the RMSE obtained on the validation set (February 2023 data).

In [52]:
!python hpo.py


100%|██████████| 15/15 [01:03<00:00,  4.24s/trial, best loss: 5.335419588556921]


C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning

In [53]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1716563281836, experiment_id='3', last_update_time=1716563281836, lifecycle_stage='active', name='random-forest-best-models', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1716561755926, experiment_id='2', last_update_time=1716561755926, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='file:///C:/Users/crab/py2/mlops-zoomcamp/cohorts/2024/02-experiment-tracking/homework/mlruns/1', creation_time=1716561207595, experiment_id='1', last_update_time=1716561207595, lifecycle_stage='active', name='homework2', tags={}>,
 <Experiment: artifact_location='file:///C:/Users/crab/py2/mlops-zoomcamp/cohorts/2024/02-experiment-tracking/homework/mlruns/0', creation_time=1716561207590, experiment_id='0', last_update_time=1716561207590, lifecycle_stage='active', name='Default', tags={}>]

In [69]:
ordered_run = client.search_runs(
    experiment_ids=2,
    filter_string='',
    max_results=1,
    order_by=['metrics.rmse ASC'])

ordered_run

[<Run: data=<RunData: metrics={'rmse': 5.335419588556921}, params={'max_depth': '19',
  'min_samples_leaf': '2',
  'min_samples_split': '2',
  'n_estimators': '11',
  'random_state': '42'}, tags={'mlflow.runName': 'upset-toad-980',
  'mlflow.source.git.commit': 'ff26d1c0c0641534b5a2f27ea1e62cb443f1a7d3',
  'mlflow.source.name': 'hpo.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'crab',
  'model': 'random forest'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/2/c2cbe3ae97d74888b921e53801d24312/artifacts', end_time=1716568475676, experiment_id='2', lifecycle_stage='active', run_id='c2cbe3ae97d74888b921e53801d24312', run_name='upset-toad-980', run_uuid='c2cbe3ae97d74888b921e53801d24312', start_time=1716568472134, status='FINISHED', user_id='crab'>, inputs=<RunInputs: dataset_inputs=[]>>]

In [64]:
ordered_run_id = 'c2cbe3ae97d74888b921e53801d24312'

order_run = client.get_run(run_id=ordered_run_id)

In [68]:
order_run.data.metrics['rmse']

5.335419588556921

What's the best validation RMSE that you got?

**5.335**

## Q6. Promote the best model to the model registry
The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

In [88]:
!python register_model.py

2024/05/25 01:20:00 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/25 01:20:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."
C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.w

In [91]:
v1_run_id = client.get_model_version(name = 'taxi-model', version =1).run_id
run1 = client.get_run(run_id=v1_run_id)

In [92]:
print(f"Version 1 Test RMSE: {(run1.data.metrics['test_rmse']):.3f}")


Version 1 Test RMSE: 5.592


What is the test RMSE of the best model?
 
5.567